In [1]:
import math
import sys

In [2]:
class TSP_Node :
    def __init__ (self, fullName=None, x=None, y=None) :
        self.fullName = None
        self.x = None
        self.y = None

    def __init__ (self, sample) : # 
        self.fullName = sample[0]
        self.x = sample[1]
        self.y = sample[2]
        
    def setData (self, fullName, x, y) :
        self.fullName = fullName
        self.x = x
        self.y = y
    def getFullName(self) :
        return self.fullName
    def getX(self) :
        return self.x
    def getY(self) :
        return self.y
    def getData (self) :
        return {
            "fullName" : self.fullName,
            "x" : self.x,
            "y" : self.y
        }
    
    def getDistanceToNode (self, node) :
        return abs(self.getX() - node.getX()) + abs(self.getY() - node.getY())
    
    def __repr__ (self) :
        return self.getFullName + ": " + str(self.getX()) + ", " + str(self.getY())


In [3]:
class TSP_Manager :
    def __init__ (self) :
        self.node_list = []
    def addNode(self, node) :
        self.node_list.append(node)
    def getNodebyIndex(self, index) :
        return self.node_list[index]
    def len_nodes(self):
        return len(self.node_list)

In [4]:
tsp_manager = TSP_Manager()

In [5]:
sample_data = [
('a', 115,61),
('b', 120,60),
('c', 125,19),
('d', 125,64),
('e', 130,64),
('f', 135,52),
('g', 215,21),
('h', 250,25),
('i', 400,63),
('j', 400,57),
('k', 400,26),
('l', 400,19),
('m', 410,61),
# ('n', 415,61),
# ('o', 425,16),
# ('p', 435,18),
# ('q', 435,23),
# ('r', 635,64),
# ('s', 645,19),
# ('t', 740,25),
# ('u', 750,25)
]

In [6]:
# init Data Array 
for data in sample_data :
    tsp_manager.addNode(TSP_Node(data))

In [7]:
for data in tsp_manager.node_list:
    print (data.fullName)

a
b
c
d
e
f
g
h
i
j
k
l
m


In [8]:
class TSP :
    def __init__(self, tsp_manager) :
        self.tsp_manager = tsp_manager
        self.graph = None
        self.maxValue = sys.maxsize
        self.lenOfNodes = self.tsp_manager.len_nodes()
        
        self.allChecked = (1 << self.lenOfNodes) - 1;
        self.memo = {}
        self.minPathDistance = 0
        self.path = []
        
    def makeGraph(self):
        lenOfNodes = self.lenOfNodes
        self.graph = [[0 for j in range(lenOfNodes+1) ] for i in range(lenOfNodes+1)]
        if (self.tsp_manager) :
            for i in range(1,lenOfNodes + 1) :
                fromNode = self.tsp_manager.getNodebyIndex(i-1)
                for j in range(i + 1, lenOfNodes + 1) :
                    toNode = self.tsp_manager.getNodebyIndex(j-1)
                    distance = fromNode.getDistanceToNode(toNode)
                    self.graph[i][j] = distance
                    self.graph[j][i] = distance
    
    def runTsp(self, fromNode, visitFlag) :
        if visitFlag == self.allChecked : # all visited
            return self.graph[fromNode][1]
        if (fromNode, visitFlag) in self.memo.keys(): # already visitFlag
            return self.memo[(fromNode, visitFlag)]

        tempValue = self.maxValue
        for next in range(1, self.lenOfNodes+1) :
            if  visitFlag & (1<<(next-1)) :
                continue
            if self.graph[fromNode][next] == 0 :
                continue
            tempValue \
            = min(tempValue, self.graph[fromNode][next] + self.runTsp(next, visitFlag | (1<<(next-1))))
        
        self.memo[(fromNode,visitFlag)] = tempValue
        return tempValue
    
    def findPath(self):
        start = 1
        visitFlag = 1
        lenOfNodes = self.lenOfNodes
        distance = self.minPathDistance
        memo = self.memo
        graph = self.graph
        path = self.path
        path.append(1)

        for i in range(1, lenOfNodes + 1):
            for j in range(1, lenOfNodes + 1):
                if (visitFlag & (1<< (j-1))):
                    print (1)
                    continue
                
                print (start, j, j, visitFlag | (1<<(j-1)) )
                cur = distance - graph[start][j]
                prev = 0
                if ((j, visitFlag | (1<<(j-1))) in memo.keys()) :
                    prev = memo[(j, visitFlag | (1<<(j-1)))]
                print ("compare", cur, prev)
                if (cur == prev ) :
                    print("a")
                    path.append(j)
                    distance = prev
                    start = j
                    visitFlag |= (1<<(j-1))
        
        print("path")
        for p in path:
            print(p)
                
    

In [9]:
tsp = TSP(tsp_manager)

In [10]:
tsp.makeGraph()

In [11]:
tsp.graph

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 6, 52, 13, 18, 29, 140, 171, 287, 289, 320, 327, 295],
 [0, 6, 0, 46, 9, 14, 23, 134, 165, 283, 283, 314, 321, 291],
 [0, 52, 46, 0, 45, 50, 43, 92, 131, 319, 313, 282, 275, 327],
 [0, 13, 9, 45, 0, 5, 22, 133, 164, 276, 282, 313, 320, 288],
 [0, 18, 14, 50, 5, 0, 17, 128, 159, 271, 277, 308, 315, 283],
 [0, 29, 23, 43, 22, 17, 0, 111, 142, 276, 270, 291, 298, 284],
 [0, 140, 134, 92, 133, 128, 111, 0, 39, 227, 221, 190, 187, 235],
 [0, 171, 165, 131, 164, 159, 142, 39, 0, 188, 182, 151, 156, 196],
 [0, 287, 283, 319, 276, 271, 276, 227, 188, 0, 6, 37, 44, 12],
 [0, 289, 283, 313, 282, 277, 270, 221, 182, 6, 0, 31, 38, 14],
 [0, 320, 314, 282, 313, 308, 291, 190, 151, 37, 31, 0, 7, 45],
 [0, 327, 321, 275, 320, 315, 298, 187, 156, 44, 38, 7, 0, 52],
 [0, 295, 291, 327, 288, 283, 284, 235, 196, 12, 14, 45, 52, 0]]

In [12]:
tsp.minPathDistance = tsp.runTsp(1, 1) 




In [13]:
tsp.minPathDistance


712

In [764]:
tsp.memo

{(1, 1): 6,
 (2, 3): 3,
 (2, 7): 3,
 (2, 11): 3,
 (3, 5): 4,
 (3, 7): 2,
 (3, 13): 4,
 (4, 9): 5,
 (4, 11): 3,
 (4, 13): 5}

In [14]:
tsp.findPath()

1
(1, 2, 2, 3)
('compare', 706, 706)
a
(2, 3, 3, 7)
('compare', 660, 662)
(2, 4, 4, 11)
('compare', 697, 699)
(2, 5, 5, 19)
('compare', 692, 700)
(2, 6, 6, 35)
('compare', 683, 683)
a
(6, 7, 7, 99)
('compare', 572, 632)
(6, 8, 8, 163)
('compare', 541, 599)
(6, 9, 9, 291)
('compare', 407, 419)
(6, 10, 10, 547)
('compare', 413, 425)
(6, 11, 11, 1059)
('compare', 392, 456)
(6, 12, 12, 2083)
('compare', 385, 451)
(6, 13, 13, 4131)
('compare', 399, 411)
1
1
(6, 3, 3, 39)
('compare', 640, 640)
a
(3, 4, 4, 47)
('compare', 595, 675)
(3, 5, 5, 55)
('compare', 590, 676)
1
(3, 7, 7, 103)
('compare', 548, 548)
a
(7, 8, 8, 231)
('compare', 509, 509)
a
(8, 9, 9, 487)
('compare', 321, 397)
(8, 10, 10, 743)
('compare', 327, 391)
(8, 11, 11, 1255)
('compare', 358, 360)
(8, 12, 12, 2279)
('compare', 353, 353)
a
(12, 13, 13, 6375)
('compare', 301, 371)
1
1
1
(12, 4, 4, 2287)
('compare', 33, 653)
(12, 5, 5, 2295)
('compare', 38, 654)
1
1
1
(12, 9, 9, 2535)
('compare', 309, 383)
(12, 10, 10, 2791)
('compar

In [15]:
tsp.path

[1, 2, 6, 3, 7, 8, 12, 11, 10, 13, 9, 5]

In [ ]:
def make_graph(datas,result):
    G = nx.DiGraph()

    for name, x, y in datas :
        G.add_node(name, pos=(x,y))

    # G.add_edges_from([('a','b')])
    for item in result:
        (y, i, j) = item[0].split('_')
        G.add_edge(chr(int(i)+97), chr(int(j)+97))


    pos=nx.get_node_attributes(G,'pos')
    nx.draw_networkx(G, pos)
    plt.show()